装包

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install gast==0.2.2
!pip install folium==0.2.1
!pip install imgaug==0.2.5
!pip install kashgari==1.1.0

  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7c71c474147168bfd1029cae9f44062c4070c2c65766cf17c6002c79f8677eca
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
     |████████████████████████████████| 71kB 1.8MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=29f336dd68094acdfad6b9801a543fcf090d9c2e0423229f5c7342cb2a7f75a9
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 563kB 2.7MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-cp36-none-any.whl size=561439 sha256=af62c91383e

数据加载 训练测试集划分

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv(r"/content/drive/My Drive/KAGGLE/nlp/twitter/data/train.csv")

In [5]:
df=df.fillna(" ")

In [6]:
import re
import string
def pre(x):
    x = re.sub('\[.*?\]', '', x)
    x = re.sub('https?://\S+|www\.\S+', '', x)
    x = re.sub('<.*?>+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), '', x)
    x = re.sub('\n', '', x)
    x = re.sub('\w*\d\w*', '', x)
    x = x.lower()
    #raw_x[i]=correct_spellings(raw_x[i])
    return x

In [7]:
cleaned_txt=df["text"].apply(pre)

In [8]:
new_feature="$ "+df['keyword']+" $ "+df["location"]+" $ "+cleaned_txt
print(new_feature.head(100))

0     $   $   $ our deeds are the reason of this ear...
1       $   $   $ forest fire near la ronge sask canada
2     $   $   $ all residents asked to shelter in pl...
3     $   $   $  people receive wildfires evacuation...
4     $   $   $ just got sent this photo from ruby a...
                            ...                        
95    $ accident $ Charlotte $  mile backup on  sout...
96    $ accident $ Baton Rouge, LA $ has an accident...
97    $ accident $ Hagerstown, MD $ breaking there w...
98    $ accident $ Gloucestershire , UK $ flowri wer...
99    $ accident $   $ only had a car for not even a...
Length: 100, dtype: object


In [9]:
'''
import re
import string
def preprocessing(raw_x):
    for i in range(len(raw_x)):
        raw_x[i]=re.sub('\[.*?\]', '', raw_x[i])
        raw_x[i] = re.sub('https?://\S+|www\.\S+', '', raw_x[i])
        raw_x[i] = re.sub('<.*?>+', '', raw_x[i])
        raw_x[i] = re.sub('[%s]' % re.escape(string.punctuation), '', raw_x[i])
        raw_x[i] = re.sub('\n', '', raw_x[i])
        raw_x[i] = re.sub('\w*\d\w*', '', raw_x[i])
        raw_x[i] = raw_x[i].lower()
        #raw_x[i]=correct_spellings(raw_x[i])
    return raw_x
'''

"\nimport re\nimport string\ndef preprocessing(raw_x):\n    for i in range(len(raw_x)):\n        raw_x[i]=re.sub('\\[.*?\\]', '', raw_x[i])\n        raw_x[i] = re.sub('https?://\\S+|www\\.\\S+', '', raw_x[i])\n        raw_x[i] = re.sub('<.*?>+', '', raw_x[i])\n        raw_x[i] = re.sub('[%s]' % re.escape(string.punctuation), '', raw_x[i])\n        raw_x[i] = re.sub('\n', '', raw_x[i])\n        raw_x[i] = re.sub('\\w*\\d\\w*', '', raw_x[i])\n        raw_x[i] = raw_x[i].lower()\n        #raw_x[i]=correct_spellings(raw_x[i])\n    return raw_x\n"

In [10]:
raw_x=new_feature.values.tolist()
raw_y=df["target"].values.tolist()
raw_data=list(zip(raw_x,raw_y))

In [11]:
training_data,test_data=train_test_split(raw_data,test_size=0.2, random_state=16)
training_data_x=[[w for w in sample[0].split()] for sample in training_data]
training_data_y=[sample[1] for sample in training_data]
test_data_x=[[w for w in sample[0].split()] for sample in test_data]
test_data_y=[sample[1] for sample in test_data]

In [12]:
import kashgari
from kashgari.embeddings import BERTEmbedding
kashgari.config.use_cudnn_cell = True

In [13]:
bert_embed = BERTEmbedding('/content/drive/My Drive/uncased_L-12_H-768_A-12/',
                           task=kashgari.CLASSIFICATION,
                           sequence_length=30)

In [14]:
from tensorflow.python import keras
import tensorflow as tf
import kashgari
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack
import logging
logging.basicConfig(level='DEBUG')
model = BiLSTM_Model(bert_embed)
eval_callback = EvalCallBack(kash_model=model,valid_x=test_data_x,valid_y=test_data_y,step=1)
model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=32,epochs=100,callbacks=[eval_callback])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 30)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 30, 768), (3 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 30, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [15]:
model.evaluate(test_data_x,test_data_y,debug_info=True)

              precision    recall  f1-score   support

           0     0.7886    0.8604    0.8229       867
           1     0.7903    0.6951    0.7397       656

    accuracy                         0.7892      1523
   macro avg     0.7894    0.7778    0.7813      1523
weighted avg     0.7893    0.7892    0.7871      1523



In [ ]:
from tensorflow.python import keras
from kashgari.tasks.classification import DPCNN_Model
import tensorflow as tf
from kashgari.callbacks import EvalCallBack
#hyper = DPCNN_Model.get_default_hyper_parameters()
#hyper['rnn_0']['units'] = 250
#hyper['rnn_1']['units'] = 250
model = DPCNN_Model(bert_embed)#,hyper_parameters=hyper)
# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=1)
early_stop = keras.callbacks.EarlyStopping(patience=100)
model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=16,epochs=20,callbacks=[eval_callback,early_stop])

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 1024),  29691904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 1024)    2048        Input-Segment[0][0]              
___________________________________________________________________________________________

In [32]:
from tensorflow.python import keras
from kashgari.tasks.classification import CNN_LSTM_Model

from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = CNN_LSTM_Model(bert_embed)

eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=1)
early_stop = keras.callbacks.EarlyStopping(patience=10)
reduse_lr_callback = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5)
# 这是 Kashgari 内置回调函数，会在训练过程计算精确度，召回率和 F1
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=test_data_x,
                             valid_y=test_data_y,
                             step=5)

model.fit(training_data_x,
          training_data_y,
          test_data_x,
          test_data_y,
          batch_size=10,epochs=30,callbacks=[eval_callback,early_stop,reduse_lr_callback])

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 30)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 30)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 30, 768), (3 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 30, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [33]:
model.evaluate(test_data_x,test_data_y,debug_info=True)

              precision    recall  f1-score   support

           0     0.7946    0.8478    0.8203       867
           1     0.7793    0.7104    0.7432       656

    accuracy                         0.7886      1523
   macro avg     0.7869    0.7791    0.7818      1523
weighted avg     0.7880    0.7886    0.7871      1523



In [34]:
df=pd.read_csv(r"/content/drive/My Drive/KAGGLE/nlp/twitter/data/test.csv")

In [35]:
df=df.fillna(" ")

In [36]:
cleaned_pre=df["text"].apply(pre)

In [37]:
pred_new_fe="$ "+df['keyword']+" $ "+df["location"]+" $ "+cleaned_pre

In [38]:
pred_new_fe.head(100)

0          $   $   $ just happened a terrible car crash
1     $   $   $ heard about earthquake is different ...
2     $   $   $ there is a forest fire at spot pond ...
3       $   $   $ apocalypse lighting spokane wildfires
4     $   $   $ typhoon soudelor kills  in china and...
                            ...                        
95    $ annihilated $   $ if your nature appropriate...
96    $ annihilated $   $ ninahoag  if you shred my ...
97    $ annihilated $ upstate NY $ thehill this is  ...
98    $ annihilated $   $ aug  ûókaiserjaegers wipe...
99    $ annihilated $   $ they should all die all of...
Length: 100, dtype: object

In [39]:
res=pred_new_fe.apply(lambda x: x.split())
res=res.tolist()

In [40]:
tested_y=model.predict(res)

In [41]:
df_fin=pd.DataFrame(zip(df['id'].tolist(),tested_y),columns=['id','target'])

In [42]:
df_fin.to_csv(r'/content/drive/My Drive/KAGGLE/nlp/twitter/res.csv')